In [4]:
!pip install torch 

In [5]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score, precision_score, f1_score

import torch 
from torch import nn
from torch import optim

import datetime

In [6]:
# Load the training and validation data

train = pd.read_csv('../public_data/train/track_a/eng.csv')
val = pd.read_csv('../public_data/dev/track_a/eng_a.csv')

train.head()

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0


In [7]:
vectorizer = CountVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform(train['text'].str.lower()).toarray()
X_val = vectorizer.transform(val['text'].str.lower()).toarray()

emotions = ['Joy','Sadness','Surprise','Fear','Anger']
y_train = train[emotions].values
y_val = val[emotions].values

print(val)

                        id                                               text  \
0    eng_dev_track_a_00001            My mouth fell open `` No, no, no... I..   
1    eng_dev_track_a_00002  You can barely make out your daughter's pale f...   
2    eng_dev_track_a_00003  But after blinking my eyes for a few times lep...   
3    eng_dev_track_a_00004  Slowly rising to my feet I came to the conclus...   
4    eng_dev_track_a_00005  I noticed this months after moving in and doin...   
..                     ...                                                ...   
111  eng_dev_track_a_00112                       "ARcH stop your progression.   
112  eng_dev_track_a_00113        This 'star', starts to move across the sky.   
113  eng_dev_track_a_00114                                  and my feet hurt.   
114  eng_dev_track_a_00115        so i cried my eyes out and did the drawing.   
115  eng_dev_track_a_00116                              They were coal black.   

     Anger  Fear  Joy  Sadn

In [ ]:
X_train_t = torch.Tensor(X_train)
y_train_t = torch.Tensor(y_train)

X_val_t = torch.Tensor(X_val)
y_val_t = torch.Tensor(y_val)

In [ ]:
print(f'Shape of X: {X_train.shape}')
print(f'Shape of y: {y_train.shape}')
print(f'Number of positives per emotion class:')
_ = [print(f' - {e}: {v} ({round(100*v/len(y_train))}%)') for e,v in zip(emotions, y_train.sum(axis=0))]

In [ ]:
model = nn.Sequential(
          nn.Linear(X_train.shape[1], 100),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

In [ ]:
# Train for a set number of epochs
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

In [ ]:
def get_predictions(X_val, model, threshold=0.5):
    sig = nn.Sigmoid() 
    yhat = sig(model(X_val)).detach().numpy()
    y_pred = yhat > threshold
    
    return y_pred

In [ ]:
y_pred = get_predictions(X_val_t, model, 0.45)
# print(y_pred)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Joy', 'Anger', 'Sadness', 'Surprise', 'Fear'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

if not 'id' in val_data_with_pred.columns:
    val_data_with_pred['id'] = val['id']
    val_data_with_pred['text'] = val['text']

val_data_with_pred = val_data_with_pred[['id', 'text', 'Joy', 'Anger', 'Sadness', 'Surprise', 'Fear']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

val_data_with_pred.to_csv(f'results/pred_eng_a.csv_{formatted_time}', index=False)

print(val_data_with_pred)

In [ ]:
def evaluate(y_val, y_pred):
    for average in ['micro', 'macro']:
        recall = recall_score(y_val, y_pred, average=average, zero_division=0)
        precision = precision_score(y_val, y_pred, average=average, zero_division=0)
        f1 = f1_score(y_val, y_pred, average=average, zero_division=0)
    
        print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

In [ ]:
# evaluate(y_val, y_pred) EVALS WON'T WORK HERE 

In [ ]:
def evaluate_per_class(y_val, y_pred):
    for i, emotion in enumerate(emotions):
        print(f'*** {emotion} ***')
    
        recall = recall_score(y_val[:,i], y_pred[:,i], zero_division=0)
        precision = precision_score(y_val[:,i], y_pred[:,i], zero_division=0)
        f1 = f1_score(y_val[:,i], y_pred[:,i], zero_division=0)
        
        print(f'recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}\n')

In [ ]:
# evaluate_per_class(y_val, y_pred) EVALS WON'T WORK HERE 

In [ ]:
weights = y_train.sum(axis=0)/y_train.sum()
weights = max(weights)/weights

In [ ]:
# Define model 
model = nn.Sequential(
          nn.Linear(X_train.shape[1], 100),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

# Define training parameters
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(weights)) # <-- weights assigned to optimiser
optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

# Train for a number of epochs
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = criterion(output, y_train_t)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

# Get predictions
y_pred = get_predictions(X_val_t, model, 0.45)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Joy', 'Anger', 'Sadness', 'Surprise', 'Fear'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

if not 'id' in val_data_with_pred.columns:
    val_data_with_pred['id'] = val['id']
    val_data_with_pred['text'] = val['text']

val_data_with_pred = val_data_with_pred[['id', 'text', 'Joy', 'Anger', 'Sadness', 'Surprise', 'Fear']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

val_data_with_pred.to_csv(f'results/pred_eng_a.csv_{formatted_time}', index=False)

print(val_data_with_pred)

# Evaluate: EVALS WON'T WORK HERE 
# print('\n\nEVALUATION\n')
# evaluate(y_val, y_pred)

# print('\nPER CLASS BREAKDOWN\n')
# evaluate_per_class(y_val, y_pred)